In [4]:
import h5py
import numpy as np

In [5]:
with h5py.File('data/deposition.h5', 'r') as ipt_deposition:
    Response_Matrix = ipt_deposition['Matrix'][...]

Response_Matrix_old = Response_Matrix       # 这是一个用于检验reshape正确性所用的语句

# Response_Matrix 
# 最终索引：(theta, phi, Det, Eng_in, Engout)
# 大小：    (7, 12, 4, 40, 100)
Response_Matrix = Response_Matrix.reshape(40, 7, 12, 4, 100)
Response_Matrix = Response_Matrix.swapaxes(0, 1).swapaxes(1, 2).swapaxes(2, 3)
Response_Matrix.shape

/tmp/ipykernel_4031/1871996035.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File('data/deposition.h5') as ipt_deposition:


(7, 12, 4, 40, 100)

In [6]:
# 用于检验矩阵分割的正确性的函数
# 最终期望结果为True(1)
# 100,000 次循环DM0U本地用时2.4s

def get_Matrix_id(id_engin, id_theta, id_phi):
    id = id_phi + id_theta * 12 + id_engin * 7 * 12
    return id

_flag = True
for _i in range(100000):
    id_engin = np.random.randint(40)
    id_theta = np.random.randint(7)
    id_phi = np.random.randint(12)
    _Bool = (Response_Matrix_old[get_Matrix_id(id_engin, id_theta, id_phi)] == \
             Response_Matrix[id_theta, id_phi, :, id_engin])
    _flag *= np.prod(_Bool)

    if _flag == False:
        break

print(_flag)


1


In [4]:
# 如果需要使用以下对拍，请将response_matrix.py与此文件置于同一目录！！！
import response_matrix as rm

In [8]:
# 双线性算法正确性的简单检验

theta_unit = np.pi / 6
phi_unit = np.pi * 2 / 12
a = rm.get_Response_Matrix(theta_unit, phi_unit / 2)
b = (Response_Matrix[1, 0] + Response_Matrix[1, 1]) / 2

bool(np.prod(a == b))

True

In [15]:
# 这是一个对拍程序，用于校验矢量化的 get_Response_Matrix_array 的正确性
# _test_time: 对拍次数
# _flag: 为True时通过检验

_test_time = 1000

Phi = np.random.random(_test_time) * 2 * np.pi
Theta = np.random.random(_test_time) * np.pi

arr_res = rm.get_Response_Matrix(Theta, Phi, 'array')

_flag = True
for i in range(_test_time):
    if np.prod(arr_res[i] == rm.get_Response_Matrix(float(Theta[i]), float(Phi[i]))) != 1:
        _flag = False
        break

print(_flag)
    

True
